In [5]:
import pandas as pd
import numpy as np
import microdf as mdf

p = pd.read_csv("../data/asec_2019_ipums.csv.gz")

# Preprocess
p.columns = p.columns.str.lower()

# Replace NIU codes.
NIU_CODES = {
    "adjginc": 99999999,
    "offtotval": 9999999999,
    # Secondary individuals under 15 have NIU for cutoff.
    # We set them to zero as to give them no UBI
    # (they're still part of SPM units with others who get UBI).
    "cutoff": 999999,
    "incwelfr": 999999,
}
for column, niu in NIU_CODES.items():
    p[column].replace({niu: 0}, inplace=True)

p["fam"] = p.marbasecidh.astype(str) + "-" + p.famid.astype(str)

# Add age flags.
p["child"] = p.age < 18
p["adult"] = ~p.child

# Create aggregates.

# Family: note families are assigned only to one SPM unit.
f = p.groupby(["fam", "cutoff", "offtotval", "spmfamunit"]).adjginc.sum().reset_index().rename(columns={"adjginc": "fam_adjginc"})

# SPM unit, with all SPM characteristics.

s = p.groupby(['spmlunch', 'spmcaphous', 'spmwt', 'spmeitc',
       'spmwic', 'spmheat', 'spmsnap', 'spmtotres', 'spmthresh', 'spmfamunit'])[
           ["incwelfr", "adult", "child"]].sum().reset_index().rename(columns={
               "incwelfr": "spmincwelfr", "adult": "spmadults", "child": "spmchildren"
               })

In [11]:
def phase_out(amount, rate, respect_to):
    return np.maximum(0, amount - respect_to * rate)


def ubi_fpg(p, f, s,
    phase_out_rate=0, respect_to="offtotval", repeal_bens=None, cutoff_ratio=1,
    adult_amount=0, child_amount=0
):
    # Branch the logic if provided at individual level.
    if adult_amount > 0:
        s = s.copy()
        s["chg"] = s.spmadults * adult_amount + s.spmchildren * child_amount
    else: 
        f["chg"] = phase_out(cutoff_ratio * f.cutoff, phase_out_rate, np.maximum(0, f[respect_to]))
        s = s.merge(f.groupby("spmfamunit").chg.sum().reset_index(), on="spmfamunit")
    if repeal_bens is not None and repeal_bens != [""]:
        s.chg -= s[repeal_bens].sum(axis=1)
    p = p.merge(s[["chg", "spmfamunit"]], on="spmfamunit")
    p.spmtotres += p.chg
    p["loser"] = p.chg < -1
    pov = mdf.poverty_rate(p, "spmtotres", "spmthresh", "asecwt")
    cost = mdf.weighted_sum(s, "chg", "spmwt")
    pct_loser = mdf.weighted_mean(p, "loser", "asecwt")
    return pd.Series({"poverty": pov, "cost": cost, "pct_loser": pct_loser})

In [15]:
sim_fam = mdf.cartesian_product({
    "phase_out": [np.inf, 0.5, 0.25, 0],
    "repeal_bens": ["", "spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch"],
    "cutoff_ratio": [1, 1.3],
    })
sim_fam = pd.concat([
    sim_fam,
    sim_fam.apply(lambda x: ubi_fpg(p, f, s, phase_out_rate=x.phase_out,
    cutoff_ratio=x.cutoff_ratio, repeal_bens=x.repeal_bens.split(",")), axis=1)
    ], axis=1)

# Individual level.
sim_ind = mdf.cartesian_product({
    "phase_out": [np.inf, 0.5, 0.25, 0],
    "adult_amount": [13300],
    "child_amount": [4516],
    "repeal_bens": ["", "spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch"]
    })
sim_ind = pd.concat([
    sim_ind,
    sim_ind.apply(lambda x: ubi_fpg(p, f, s, phase_out_rate=x.phase_out,
    adult_amount=x.adult_amount, child_amount=x.child_amount, repeal_bens=x.repeal_bens.split(",")), axis=1)
    ], axis=1)

sim = pd.concat([sim_fam, sim_ind])

cur_pov = sim[(sim.phase_out == np.inf) & (sim.repeal_bens == "")].poverty.values[0]

def pct_chg(base, new):
    return (new - base) / base

sim["pov_chg"] = pct_chg(cur_pov, sim.poverty)
sim

,phase_out,repeal_bens,cutoff_ratio,poverty,cost,pct_loser,adult_amount,child_amount,pov_chg
0,inf,,1.0,0.117132,0.000000e+00,0.000000,NaN,NaN,0.000000
1,inf,,1.3,0.117132,0.000000e+00,0.000000,NaN,NaN,0.000000
2,inf,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",1.0,0.150437,-1.075703e+11,0.399615,NaN,NaN,0.284336
3,inf,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",1.3,0.150437,-1.075703e+11,0.399615,NaN,NaN,0.284336
4,0.50,,1.0,0.031919,3.442652e+11,0.000000,NaN,NaN,-0.727499
5,0.50,,1.3,0.012308,6.101787e+11,0.000000,NaN,NaN,-0.894922
6,0.50,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",1.0,0.047664,2.366949e+11,0.235764,NaN,NaN,-0.593073
7,0.50,"spmeitc,spmcaphous,spmsnap,spmincwelfr,spmlunch",1.3,0.016307,5.026084e+11,0.174073,NaN,NaN,-0.860780
8,0.25,,1.0,0.018187,7.552311e+11,0.000000,NaN,NaN,-0.844727
9,0.25,,1.3,0.006343,1.267263e+12,0.000000,NaN,NaN,-0.945846
